<a href="https://colab.research.google.com/github/FairozAhmadSheikh/DeepLearning_Bootcamp_with_Tensor_Flow/blob/main/Mini_Food_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip

--2025-10-02 09:48:40--  https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.135.207, 142.250.107.207, 142.251.188.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.135.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1625420029 (1.5G) [application/zip]
Saving to: ‘101_food_classes_10_percent.zip.1’

101_food_classes_10 100%[===================>]   1.51G   114MB/s    in 9.8s    

2025-10-02 09:48:50 (158 MB/s) - ‘101_food_classes_10_percent.zip.1’ saved [1625420029/1625420029]



In [3]:
!wget https://raw.githubusercontent.com/FairozAhmadSheikh/DeepLearning_Bootcamp_with_Tensor_Flow/refs/heads/main/helper_functions/helper_functions.py

--2025-10-02 09:48:50--  https://raw.githubusercontent.com/FairozAhmadSheikh/DeepLearning_Bootcamp_with_Tensor_Flow/refs/heads/main/helper_functions/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6733 (6.6K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]   6.58K  --.-KB/s    in 0s      

2025-10-02 09:48:50 (81.4 MB/s) - ‘helper_functions.py.1’ saved [6733/6733]



In [4]:
from helper_functions import create_tensorboard_callback,plot_loss_curves,compare_historys,walk_through_dir,unzip_data

In [5]:
unzip_data('101_food_classes_10_percent.zip')

In [6]:
walk_through_dir('101_food_classes_10_percent')

There are 2 directories and 0 images in '101_food_classes_10_percent'.
There are 101 directories and 0 images in '101_food_classes_10_percent/train'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/falafel'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/ravioli'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/escargots'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/lobster_bisque'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/crab_cakes'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/spaghetti_carbonara'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/pancakes'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/red_velvet_cake'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/edamame'.
There are 0 directories and 75 images in '1

In [7]:
train_dir='101_food_classes_10_percent/train'
test_dir='101_food_classes_10_percent/test'

In [8]:
import tensorflow as tf

In [9]:
IMG_SIZE=(224,224)

In [10]:
train_all_10_percent=tf.keras.preprocessing.image_dataset_from_directory(train_dir,label_mode='categorical',image_size=IMG_SIZE)
test_data=tf.keras.preprocessing.image_dataset_from_directory(test_dir,label_mode='categorical',image_size=IMG_SIZE,shuffle=False)

Found 7575 files belonging to 101 classes.
Found 25250 files belonging to 101 classes.


Create a model checkpoint callback

In [11]:
chechpoint_path='101_food_all_data_10%_checkpoint.weights.h5'
check_point_callback=tf.keras.callbacks.ModelCheckpoint(chechpoint_path,save_weights_only=True,monitor='val_accuracy',save_best_only=True)

# **Create Data Augmentation Layer**

In [12]:
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras import preprocessing

In [13]:
data_augmentation=Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomHeight(0.2),
    layers.RandomWidth(0.2),
    layers.RandomRotation(0.2),
    # layers.Rescaling(1/255.)
],name='data_augmentation_layer')

In [14]:
# Setup base model
base_model=tf.keras.applications.EfficientNetB0(include_top=False,weights='imagenet')
base_model.trainable=False

In [15]:
# Setup model architecture with trainable top layer
inputs=layers.Input(shape=(224,224,3),name='input_layer')
x=data_augmentation(inputs)
x=base_model(x,training=False)
x=layers.GlobalAveragePooling2D(name='Global_Average_pooling_layer')(x)
outputs=layers.Dense(len(train_all_10_percent.class_names),activation='softmax',name='Output_layer')(x)

model=tf.keras.Model(inputs,outputs)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation_layer         │ (None, None, None, 3)  │             0 │
│ (Sequential)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, None, None,     │     4,049,571 │
│                                 │ 1280)                  │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Global_Average_pooling_layer    │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output_layer (Dense)            │ (None, 101)            │       129,381 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,178,952 (15.94 MB)

 Trainable params: 129,381 (505.39 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [16]:
# Compile and fit
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

Model Training

In [ ]:
history_all_classes_10_percent=model.fit(train_all_10_percent,epochs=5,validation_data=test_data,validation_steps=int(0.15*len(test_data)),callbacks=[check_point_callback])

Epoch 1/5
237/237 ━━━━━━━━━━━━━━━━━━━━ 56s 174ms/step - accuracy: 0.1743 - loss: 3.9024 - val_accuracy: 0.3861 - val_loss: 2.6270
Epoch 2/5
 85/237 ━━━━━━━━━━━━━━━━━━━━ 18s 119ms/step - accuracy: 0.4811 - loss: 2.3099

In [ ]:
plot_loss_curves(history_all_classes_10_percent)

In [ ]:
# Unfreeze some layers for fine tunning the
base_model.trainable=True

for layer  in base_model.layers[:-5]:
  layer.trainable=False

In [ ]:
# Check which layer in model are trainable
for layer_no ,layer in enumerate(model.layers[2].layers):
  print(layer_no,layer.name,layer.trainable)

In [ ]:
for layer in model.layers:
  print(layer.name,layer.trainable)

In [ ]:
# Recompile the model

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])
# Model Training
fine_tune_epochs=10

history_all_classes_10_percent_fine_tuned=model.fit(train_all_10_percent,
                                                    epochs=fine_tune_epochs,
                                                    initial_epoch=history_all_classes_10_percent.epoch[-1],
                                                    validation_data=test_data,
                                                    validation_steps=int(0.15*len(test_data)))

In [ ]:
all_classes_10_percent_fine_tune_result=model.evaluate(test_data)

Compare Histories

In [ ]:
from types import new_class
compare_historys(original_history=history_all_classes_10_percent,new_history=history_all_classes_10_percent_fine_tuned,initial_epochs=5)

Save and Load Model

In [ ]:
model_paths="/content/drive/MyDrive/deep_learning_models"
model.save(filepath=model_paths+"/"+"fine_tuned_model_all_data_10_percent.h5")

In [ ]:
# Define the custom objects dictionary
custom_objects = {"RandomHeight": tf.keras.layers.RandomHeight,
                  "RandomWidth": tf.keras.layers.RandomWidth,
                  "RandomRotation": tf.keras.layers.RandomRotation,
                  "RandomFlip": tf.keras.layers.RandomFlip}

# Load the model with custom objects
loaded_model = tf.keras.models.load_model("/content/drive/MyDrive/deep_learning_models/fine_tuned_model_all_data_10_percent.h5",
                                        custom_objects=custom_objects)

loaded_model.summary()

In [ ]:
loaded_model_results=loaded_model.evaluate(test_data)

Evaluating model performance accross differnt classes

In [ ]:
import tensorflow as tf

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/custom_food_images.zip

Making predictions with our trained_model

In [ ]:
pred_probs=model.predict(test_data,verbose=1)

In [ ]:
pred_probs

In [ ]:
len(pred_probs)

In [ ]:
pred_probs.shape

In [ ]:
pred_probs[0]

In [ ]:
len(pred_probs[0])

In [ ]:
sum(pred_probs[0])

In [ ]:
tf.argmax(pred_probs[0])

In [ ]:
test_data.class_names

In [ ]:
test_data.class_names[52]

In [ ]:
# Get predicted classes of each label
pred_classes=pred_probs.argmax(axis=1)
pred_classes[:10]

In [ ]:
test_data

In [ ]:
len(test_data)   # return number of batches

To get our test labels we need ti unravel our test_data batchdataset

In [ ]:
y_labels=[]
for images ,labels in test_data.unbatch():
  y_labels.append(labels.numpy().argmax())
y_labels[:10]

In [ ]:
loaded_model_results

Evaluating our models predictions

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
sklearn_accuracy=accuracy_score(y_true=y_labels,y_pred=pred_classes)

In [ ]:
sklearn_accuracy

Lets now visualize (Make Confusion Matrix)

In [ ]:
from helper_functions import make_confusion_matrix

In [ ]:
class_names=test_data.class_names

In [ ]:
make_confusion_matrix(y_true=y_labels,y_pred=pred_classes,classes=class_names,figsize=(100,100),text_size=(20))

Lets create classification report

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_labels,pred_classes))

Creating a Dictionary of classification report

In [ ]:
classification_report_dict=classification_report(y_labels,pred_classes,output_dict=True)

In [ ]:
# Create Empty Dictinary first
class_f1_scores={}

for k,v in classification_report_dict.items():
   if k == 'accuracy':
    break
   else:
      class_f1_scores[class_names[int(k)]]=v['f1-score']
class_f1_scores

In [ ]:
# Turn f1 scores into dataframe visulzization
import pandas as pd

f1_score_df= pd.DataFrame({
    'class_names': list(class_f1_scores.keys()),
    "f1-score": list(class_f1_scores.values())
}).sort_values('f1-score', ascending=False)

display(f1_score_df)

# **Plot F1-Scores**

In [ ]:
import matplotlib.pyplot as plt
fig,ax=plt.subplots(figsize=(12,25))
scores=ax.barh(range(len(f1_score_df)),f1_score_df['f1-score'].values)
ax.set_yticks(range(len(f1_score_df)))
ax.set_yticklabels(f1_score_df['class_names'])
ax.set_xlabel('f1-score')
ax.set_title('f1-scores for 101 classes')
ax.invert_yaxis()

# **Visualzize predictions on custom images**

In [ ]:
def load_and_prep_image(filename,img_shape=224,scale=True):
    img=tf.io.read_file(filename)
    img=tf.io.decode_image(img)
    img=tf.image.resize(img,[img_shape,img_shape])

    if scale:
       return img/255.
    else:
       return img

In [ ]:
import os
import random
import matplotlib.pyplot as plt

plt.figure(figsize=(17,10))
for i in range(3):
    class_name=random.choice(class_names)
    file_name=random.choice(os.listdir(test_dir+'/'+class_name))
    # Fix: Correct the file path to include the class_name directory
    file_path=test_dir+'/'+class_name+'/'+file_name
    # Load and make prediction
    img=load_and_prep_image(file_path,scale=False)
    pred_probs=model.predict(tf.expand_dims(img,axis=0))
    pred_class=class_names[pred_probs.argmax()]

    # Plot Images
    plt.subplot(1,3,i+1)
    plt.imshow(img/255.)
    if class_name==pred_class:
       title_color='g'
    else:
       title_color='r'
    plt.title(f'Actual:{class_name}, pred:{pred_class} probability {pred_probs.max():.2f}',c=title_color)
    plt.axis(False)

***Finding the most wrong predictions***

In [ ]:
# Get all of the image filepaths in the test datasets
filepaths=[]
for filepath in test_data.list_files('101_food_classes_10_percent/test/*/*.jpg',shuffle=False):
    filepaths.append(filepath.numpy())
filepaths[:10]

In [ ]:
#2 create a DataFrame of different parameters for each of our test images
import pandas as pd
import numpy as np

# Collect filepaths, true labels, and predicted classes by iterating through the dataset
filepaths = []
y_labels = []
# Ensure pred_classes is a numpy array for easier indexing
pred_classes = np.argmax(model.predict(test_data, verbose=0), axis=1)

for filepath, labels in test_data.unbatch():
    filepaths.append(filepath.numpy())
    y_labels.append(labels.numpy().argmax())

pred_df=pd.DataFrame({'img_path':filepaths,
                     "y_true":y_labels,
                     "y_pred":pred_classes,
                     'pred_conf':pred_probs.max(axis=1),
                     'y_true_classnames':[class_names[i] for i in y_labels],
                     'y_pred_classnames':[class_names[i] for i in pred_classes]
                      })
display(pred_df.head(3))

In [ ]:
pred_df

In [ ]:
# find out in our dataframe which predictions are wrong
pred_df['pred_correct']=pred_df['y_true']==pred_df['y_pred']
pred_df.head(3)

In [ ]:
# Sort our Dataframe to have most wrong predictions at top
top_100_wrong=pred_df[pred_df['pred_correct']==False].sort_values('pred_conf',ascending=False)[:100]
top_100_wrong.head()

In [ ]:
# Visualize the test data samples which have the wrong prediction but high probability

images_to_view=9
start_index=10
plt.figure(figsize=(15,10))
for i , row in enumerate(top_100_wrong[start_index:start_index+images_to_view].itertuples()):
  plt.subplot(3,3,i+1)
  # Access elements from the row tuple using their names
  y_true = row.y_true
  y_pred = row.y_pred
  pred_prob = row.pred_conf
  # Get class names using the indices
  y_true_classname = class_names[y_true]
  y_pred_classname = class_names[y_pred]
  # Load the image using the img_path from the original pred_df
  img_path = pred_df.loc[row.Index, 'img_path']
  img = load_and_prep_image(img_path, scale=False)

  plt.imshow(img/255.)
  plt.title(f'actual :{y_true_classname},pred:{y_pred_classname} \n prob {pred_prob:.2f}')
  plt.axis(False)